In [1]:
import torch
import numpy as np
from smplx import SMPLX
from smotdm.renderer.matplotlib import MatplotlibRender
from smotdm.utils import loop_interx
from smotdm.rifke import joints_to_rifke, rifke_to_joints


INTERX_DATASET_ROOT = 'deps/interx'
device = torch.device("cpu")

renderer = MatplotlibRender(jointstype='smplxjoints')

In [2]:
iterator = loop_interx(INTERX_DATASET_ROOT)


for scene_id, num_frames, motions, texts in iterator:
    smplx_model = SMPLX(
        model_path='deps/smplx/SMPLX_NEUTRAL.npz',
        batch_size=num_frames,
        num_betas=10,
        use_pca=False,
        use_face_contour=True
    )
    # joints[..., 0] *= -1
    # joints_m = swap_left_right(joints)

    feats = []
    for motion in motions:
        output = smplx_model(
            **motion,
            return_full_pose=True,
        )
        # XZY -> XYZ
        joints = torch.from_numpy(
            np.dot(output.joints.detach().cpu().numpy(), np.array(
                [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]]
            ))
        )

        feats.append(joints_to_rifke(joints[:, :smplx_model.NUM_JOINTS, :]))
    scene_motions = torch.stack(feats)
    break

scene_motions.shape

  0%|          | 0/11388 [00:00<?, ?it/s]

torch.Size([2, 1456, 163])

In [3]:
texts

["A person holds the other person's left arm with his/her own hands and then uses his/her right hand to hold his/her left arm and walk forward with him/her.",
 'One person sits and touches his/her head with his/her right hand, while the other person puts his/her arms around his/her left arm, pulls him/her up, and walks forward together.',
 'One person helps the other sitting man up by taking his left hand with both arms.']

In [5]:
import h5py
d = h5py.File('t.h5', 'w')
d.create_dataset('123', data=scene_motions[:, :num_frames, :].detach().cpu().numpy())
d.close()

In [4]:
renderer(rifke_to_joints(scene_motions[0]).detach().cpu().numpy(), rifke_to_joints(scene_motions[1]).detach().cpu().numpy(), output="test.mp4")